In [1]:
# Taken from https://github.com/Microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('./regression.train.txt', header=None, sep='\t')
df_test = pd.read_csv('./regression.test.txt', header=None, sep='\t')

y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

/Users/nbarraille/dev/blazee/blazee-examples/.venv/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


Loading data...
Starting training...
[1]	valid_0's l1: 0.491812	valid_0's l2: 0.24288
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.48798	valid_0's l2: 0.239307
[3]	valid_0's l1: 0.483905	valid_0's l2: 0.235559
[4]	valid_0's l1: 0.478533	valid_0's l2: 0.230771
[5]	valid_0's l1: 0.47333	valid_0's l2: 0.226297
[6]	valid_0's l1: 0.470079	valid_0's l2: 0.223692
[7]	valid_0's l1: 0.466768	valid_0's l2: 0.220941
[8]	valid_0's l1: 0.462971	valid_0's l2: 0.217982
[9]	valid_0's l1: 0.459474	valid_0's l2: 0.215351
[10]	valid_0's l1: 0.45619	valid_0's l2: 0.213064
[11]	valid_0's l1: 0.4532	valid_0's l2: 0.211053
[12]	valid_0's l1: 0.450526	valid_0's l2: 0.209336
[13]	valid_0's l1: 0.447621	valid_0's l2: 0.207492
[14]	valid_0's l1: 0.445178	valid_0's l2: 0.206016
[15]	valid_0's l1: 0.442718	valid_0's l2: 0.204677
[16]	valid_0's l1: 0.440293	valid_0's l2: 0.203224
[17]	valid_0's l1: 0.437015	valid_0's l2: 0.201186
[18]	valid_0's l1: 0.434725	valid_0's l2: 0.199626

In [9]:
import blazee
import os
import logging
logging.basicConfig(level=logging.INFO)
%load_ext autoreload
%autoreload 2

# Deploy the model on Blazee
api_key = os.environ['BLAZEE_API_KEY']
bz = blazee.Blazee(api_key)
bm = bz.deploy_model(gbm, model_name="LightGBM Regressor")
bm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:root:Uploading model version to Blazee  (53.2 KB)...
INFO:root:Deploying new model version: v1...
INFO:root:Successfully deployed model version 67c3f90b-66f1-482e-a1af-ca3d2220f0b3


<BlazeeModel 'LightGBM Regressor'
	id=21dd16c9-fc21-4460-b0e5-a938a079f5b4>

In [15]:
bm.predict_batch(X_test)

[<Prediction
 	prediction=0.5947353974438443
 	probas=None>, <Prediction
 	prediction=0.45111956341621295
 	probas=None>, <Prediction
 	prediction=0.3480660664395888
 	probas=None>, <Prediction
 	prediction=0.5088970423738973
 	probas=None>, <Prediction
 	prediction=0.3642672514850198
 	probas=None>, <Prediction
 	prediction=0.33970103351449715
 	probas=None>, <Prediction
 	prediction=0.441384496557629
 	probas=None>, <Prediction
 	prediction=0.3660781792869375
 	probas=None>, <Prediction
 	prediction=0.6959806061667265
 	probas=None>, <Prediction
 	prediction=0.4082073787837169
 	probas=None>, <Prediction
 	prediction=0.6000691217623922
 	probas=None>, <Prediction
 	prediction=0.7198000750954558
 	probas=None>, <Prediction
 	prediction=0.6928338119440839
 	probas=None>, <Prediction
 	prediction=0.68357819033879
 	probas=None>, <Prediction
 	prediction=0.4742792119425314
 	probas=None>, <Prediction
 	prediction=0.6535297374127085
 	probas=None>, <Prediction
 	prediction=0.3964201510467

In [ ]:
gbm.predict(X_test)